In [1]:
from dotenv import load_dotenv
load_dotenv()

from crewai import LLM
llm = LLM(
    model = "gemini/gemini-2.5-flash-lite",
    temperature=0.1
)

In [2]:
from crewai.tools import BaseTool
from serpapi import GoogleSearch
import os

class SerpAPITool(BaseTool):
    name: str = "Search Internet with SerpAPI"
    description: str = "Search the internet for latest information"

    def _run(self, query: str):
        params = {
            "engine": "google",
            "q": query,
            "api_key": os.getenv("SERPAPI_API_KEY"),
            "num": 5
        }

        search = GoogleSearch(params)
        results = search.get_dict()

        output = ""

        if "organic_results" in results:
            for result in results["organic_results"]:
                title = result.get("title", "")
                link = result.get("link", "")
                snippet = result.get("snippet", "")
                output += f"\nTitle: {title}\nLink: {link}\nSnippet: {snippet}\n"

        return output

In [4]:
from crewai import Agent , Task , Crew


research_agent = Agent(
    role = "Research specialist",
    goal = "research the latest interesting updates and facts about {topic} and summarize them in a report",
    backstory = "you are very expert and have deep knowledge in research and you are very good to find out letest updates and facts aout any topic",
    tools = [SerpAPITool()],
    verbose = True,
    llm = llm
)

writing_agent = Agent(
    role = "Writing specialist",
    goal = "write a report about the latest interesting updates and facts about {topic} based on the research provided by the research agent",
    backstory = "you are skilled writer and very good to write a report base on the research provided by the research agent and you are very good to write a report in a clear and concise way",
    verbose = True,
    llm = llm
)

research_task = Task(
    description = "do fully professional research about {topic} and also tell about the source of your research",
    expected_output="give a professional output about the letest interesting updates and facts about {topic} in proper report format",
    agent = research_agent
)

writer_task = Task(
    description="write a fully professional report for {topic} base on the research provided by the research agent",
    expected_output="a fully professional report formatted about letest updates and facts for {topic}",
    agent=writing_agent,
    context=[research_task],

)

crew = Crew(
    agents = [research_agent , writing_agent],
    tasks = [research_task , writer_task],
    verbose = True

)
result  = crew.kickoff(inputs={"topic": "Future of AI(Artificial Intelligence)"})
print(result)

╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  8e5f42d8-2a01-4e97-8b09-e31641104071                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name: do fully professional research about Future of AI(Artificial Intelligence) and also tell about the       │
│  source of your research                                                                                        │
│  ID: d03d2eb2-2443-4f59-b205-29a571364df4                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research specialist                                                                                     │
│                                                                                                                 │
│  Task: do fully professional research about Future of AI(Artificial Intelligence) and also tell about the       │
│  source of your research                                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── 🔧 Tool Execution Started (#1) ─────────────────────────────────────────╮
│                                                                                                                 │
│  Tool: search_internet_with_serp_api                                                                            │
│  Args: {'query': 'Future of AI latest updates and facts'}                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Tool search_internet_with_serp_api executed with result: 
Title: What's next in AI: 7 trends to watch in 2026
Link: https://news.microsoft.com/source/features/ai/whats-next-in-ai-7-trends-to-watch-in-2026/
Snippet: Seven AI trends to watch in 2026 will make...


...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Writing specialist                                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ## Report: The Future of Artificial Intelligence                                                               │
│                                                                                                                 │
│  **Date:** October 26, 2023                                                                                     │
│                                                                                                                 │
│  **Prepared For:** Interested Stakeholders                                                                      │
│  **Prepared By:** [Your Name/Department]                                                                        │
│                                                                                                                 │
│  **1. Executive Summary**                                                                                       │
│                                                                                                                 │
│  Artificial Intelligence (AI) is undergoing a period of rapid and transformative evolution, poised to           │
│  fundamentally alter industries and everyday human experiences. This report synthesizes recent key              │
│  developments and trends, highlighting AI's progression towards becoming a collaborative partner, significant   │
│  advancements in its performance capabilities, the ongoing challenges and successes in its adoption, and the    │
│  critical role of hardware innovation in supporting its growth. The overarching trajectory indicates a future   │
│  where AI is more deeply integrated, more capable, and increasingly essential across a broad spectrum of        │
│  applications.                                                                                                  │
│                                                                                                                 │
│  **2. Introduction**                                                                                            │
│                                                                                                                 │
│  The field of Artificial Intelligence continues to advance at an unprecedented pace, moving beyond theoretical  │
│  concepts to practical, impactful applications. This report aims to provide a comprehensive overview of the     │
│  most significant recent updates and facts shaping the future of AI. By examining key trends, performance       │
│  benchmarks, adoption rates, and underlying technological enablers, this document offers insights into the      │
│  evolving landscape of AI and its projected impact.                                                             │
│                                                                                                                 │
│  **3. Key Trends and Developments**                                                                             │
│                                                                                                                 │
│  **3.1. AI as a Collaborative Partner**                                                                         │
│                                                                                                                 │
│  A significant shift is anticipated in the role of AI, 

╭────────────────────────────────────────────── 📋 Task Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name:                                                                                                          │
│  write a fully professional report for Future of AI(Artificial Intelligence) base on the research provided by   │
│  the research agent                                                                                             │
│  Agent:                                                                                                         │
│  Writing specialist                                                                                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  8e5f42d8-2a01-4e97-8b09-e31641104071                                                                           │
│  Final Output: ## Report: The Future of Artificial Intelligence                                                 │
│                                                                                                                 │
│  **Date:** October 26, 2023                                                                                     │
│                                                                                                                 │
│  **Prepared For:** Interested Stakeholders                                                                      │
│  **Prepared By:** [Your Name/Department]                                                                        │
│                                                                                                                 │
│  **1. Executive Summary**                                                                                       │
│                                                                                                                 │
│  Artificial Intelligence (AI) is undergoing a period of rapid and transformative evolution, poised to           │
│  fundamentally alter industries and everyday human experiences. This report synthesizes recent key              │
│  developments and trends, highlighting AI's progression towards becoming a collaborative partner, significant   │
│  advancements in its performance capabilities, the ongoing challenges and successes in its adoption, and the    │
│  critical role of hardware innovation in supporting its growth. The overarching trajectory indicates a future   │
│  where AI is more deeply integrated, more capable, and increasingly essential across a broad spectrum of        │
│  applications.                                                                                                  │
│                                                                                                                 │
│  **2. Introduction**                                                                                            │
│                                                                                                                 │
│  The field of Artificial Intelligence continues to advance at an unprecedented pace, moving beyond theoretical  │
│  concepts to practical, impactful applications. This report aims to provide a comprehensive overview of the     │
│  most significant recent updates and facts shaping the future of AI. By examining key trends, performance       │
│  benchmarks, adoption rates, and underlying technological enablers, this document offers insights into the      │
│  evolving landscape of AI and its projected impact.                                                             │
│                                                                                                                 │
│  **3. Key Trends and Developments**                                                                             │
│                                                                                                                 │
│  **3.1. AI as a Collaborative Partner**               

## Report: The Future of Artificial Intelligence

**Date:** October 26, 2023

**Prepared For:** Interested Stakeholders
**Prepared By:** [Your Name/Department]

**1. Executive Summary**

Artificial Intelligence (AI) is undergoing a period of rapid and transformative evolution, poised to fundamentally alter industries and everyday human experiences. This report synthesizes recent key developments and trends, highlighting AI's progression towards becoming a collaborative partner, significant advancements in its performance capabilities, the ongoing challenges and successes in its adoption, and the critical role of hardware innovation in supporting its growth. The overarching trajectory indicates a future where AI is more deeply integrated, more capable, and increasingly essential across a broad spectrum of applications.

**2. Introduction**

The field of Artificial Intelligence continues to advance at an unprecedented pace, moving beyond theoretical concepts to practical, impactful appli

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Research agent -> internet -> after find from intrner back to reaserch and research agent make research -> writing agent > crew